In [1]:
def timestamps(total_time, split):
    iters = int(total_time/split)
    for i in range(1, iters + 1):
        yield (i-1)*split, i * split, i

In [9]:
from pydub import AudioSegment
import pathlib
import os

pathlib.Path('audios/roganelon').mkdir(parents=True, exist_ok=True)

newAudio = AudioSegment.from_wav("audios/roganelon.wav")

for start, end, val in timestamps(len(newAudio), 5000):
    cutAudio = newAudio[start:end]
    cutAudio.export(f'audios/roganelon/roganelon{val-1}.wav', format='wav')

In [10]:
header = 'filename chroma_stft rmse spectral_centroid spectral_bandwidth rolloff zero_crossing_rate'
for i in range(1, 21):
    header += f' mfcc{i}'
header += ' label'
header = header.split()

In [11]:
import csv
import numpy as np
import os
import librosa



file = open('roganelon_dataset.csv', 'w', newline='')

with file:
    writer = csv.writer(file)
    writer.writerow(header)

for p in ['roganelon']:
    for filename in os.listdir(f'audios/{p}'):
        songname = f'audios/{p}/{filename}'
        y, sr = librosa.load(songname, mono=True, duration=5)
        
        rmse = librosa.feature.rms(y=y) #root mean squred ENERGY note. changed to rms form rmse
        chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
        spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
        spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
        rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
        zcr = librosa.feature.zero_crossing_rate(y)
        mfcc = librosa.feature.mfcc(y=y, sr=sr)
        
        to_append = f'{filename} {np.mean(chroma_stft)} {np.mean(rmse)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}'
        
        for e in mfcc:
            to_append += f' {np.mean(e)}'
        
        to_append += f' {p}'
        
        file = open('roganelon_dataset.csv', 'a', newline='')
        with file:
            writer = csv.writer(file)
            writer.writerow(to_append.split())